# Using Open-source HuggingFace Models (Based on Gemma-7b-it) to Generate QAs from Raw Data in JSON format

In this example, we will show you how to generate question-answers (QAs) from given text strings using open-source Huggingface models **using [Gemma-7b-it](https://huggingface.co/google/gemma-7b-it)** via uniflow's [HuggingFaceModelFlow](https://github.com/CambioML/uniflow/blob/main/uniflow/flow/model_flow.py#L86).

### Before running the code

You will need to `uniflow` conda environment to run this notebook. You can set up the environment following the instruction: https://github.com/CambioML/uniflow/tree/main#installation.

Next, you will need a valid Huggingface Token to access the Gemma model. Once you have the token, set it as the environment variable `HF_TOKEN` within a `.env` file in the root directory of this repository. Check [this](https://huggingface.co/docs/huggingface_hub/en/package_reference/environment_variables) for more details.

### Update system path

In [1]:
%reload_ext autoreload
%autoreload 2

import sys

sys.path.append(".")
sys.path.append("..")
sys.path.append("../..")

### Install libraries

In [2]:
!{sys.executable} -m pip install -q transformers accelerate bitsandbytes scipy tiktoken

### Import dependency

In [3]:
from pprint import pprint
from dotenv import load_dotenv
from IPython.display import Markdown, display

from uniflow.flow.client import TransformClient
from uniflow.flow.config import GemmaTransformConfig
from uniflow.op.prompt import Context, PromptTemplate

load_dotenv()

/opt/conda/envs/uniflow/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

### Prepare sample prompts

First, we need to demonstrate sample prompts for LLM, those include instruction and sample json format. We do this by giving a sample instruction and list of `Context` examples to the `PromptTemplate` class. Let's create raw text for context.

In [4]:
raw_context_input = [
    """We believe our success depends upon our capabilities in areas such as design, research and development, \
production and marketing and is supported and protected by our intellectual property rights, such as \
trademarks, utility and design patents, copyrights, and trade secrets, among others. We have followed a policy \
of applying for and registering intellectual property rights in the United States and select foreign countries \
on trademarks, inventions, innovations and designs that we deem valuable. W e also continue to vigorously \
protect our intellectual property, including trademarks, patents and trade secrets against third-party \
infringement and misappropriation.""",
    """Snoopy can be selfish, gluttonous, and lazy at times, and occasionally mocks his owner, Charlie Brown. 
But on the whole, he shows great love, care, and loyalty for his owner (even though he cannot even remember 
his name and always refers to him as "the round-headed kid"). In the 1990s comic strips, he is obsessed 
with cookies, particularly the chocolate-chip variety. This, and other instances in which he indulges in large 
chocolate-based meals and snacks, indicate that chocolate is not poisonous to Snoopy, the way it is for real dogs.""",
    """Since the 2000s and especially the late 2010s, Intel has faced increasing competition from AMD, resulting in a 
significant decline of its dominance and market share in the PC market. Nevertheless, with a 68.4% market share as of 
2023, Intel still leads the x86 market by a wide margin."""
]

raw_context_input = raw_context_input

Next, for the given raw text strings `raw_context_input` above, we convert them to the `Context` class to be processed by `uniflow`.

In [5]:

input_data = [
    Context(context=data)
    for data in raw_context_input
]

print("sample size of processed input data: ", len(input_data))

print("\nExample uniflow context data:")
pprint(input_data[:3])


sample size of processed input data:  3

Example uniflow context data:
[Context(context='We believe our success depends upon our capabilities in areas such as design, research and development, production and marketing and is supported and protected by our intellectual property rights, such as trademarks, utility and design patents, copyrights, and trade secrets, among others. We have followed a policy of applying for and registering intellectual property rights in the United States and select foreign countries on trademarks, inventions, innovations and designs that we deem valuable. W e also continue to vigorously protect our intellectual property, including trademarks, patents and trade secrets against third-party infringement and misappropriation.'),
 Context(context='Snoopy can be selfish, gluttonous, and lazy at times, and occasionally mocks his owner, Charlie Brown. \nBut on the whole, he shows great love, care, and loyalty for his owner (even though he cannot even remember \nhis 

### Use LLM to generate data

In this example, we will use the [GemmaTransformConfig](https://github.com/CambioML/uniflow/blob/main/uniflow/flow/config.py)'s default LLM to generate questions and answers. Let's import the config and client of this model.

Here in this example, we use define our own `PromptTemplate` to the `GemmaTransformConfig`, but you can use your default instructions and examples instead if you want.


In [6]:
guided_prompt = PromptTemplate(
        instruction="Generate one question and its corresponding answer based on context. Following the format and structure of the examples below to include the same context, question, and answer in the response.",
        few_shot_prompt=[
            Context(
                context="In 1948, Claude E. Shannon published A Mathematical Theory of\nCommunication (Shannon, 1948) establishing the theory of\ninformation. In his article, Shannon introduced the concept of\ninformation entropy for the first time. We will begin our journey here.",
                question="Who published A Mathematical Theory of Communication in 1948?",
                answer="Claude E. Shannon.",
            )
        ]
)

In [7]:
config = GemmaTransformConfig(
    prompt_template=guided_prompt,
)

client = TransformClient(config)

tokenizer_config.json: 100%|██████████| 2.16k/2.16k [00:00<00:00, 15.6MB/s]
tokenizer.model: 100%|██████████| 4.24M/4.24M [00:00<00:00, 73.3MB/s]
tokenizer.json: 100%|██████████| 17.5M/17.5M [00:00<00:00, 219MB/s]
config.json: 100%|██████████| 694/694 [00:00<00:00, 5.47MB/s]
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
model.safetensors.index.json: 100%|██████████| 20.9k/20.9k [00:00<00:00, 109MB/s]
model-00001-of-00004.safetensors: 100%|██████████| 5.00G/5.00G [00:21<00:00, 233MB/s]
model-00002-of-00004.safetensors: 100%|██████████| 4.98G/4.98G [00:28<00:00, 174MB/s]
model-00003-of-00004.safetensors: 100%|██████████| 4.98G/4.98G [00:31<00:00, 159MB/s]
model-00004-of-00004.safetensors: 100%|██████████| 2.11G/2.11G [00:15<00:00, 133MB/s]
generation_config.json: 100%|██████████| 137/137 [00:00<00:00, 987kB/s]


Now we call the `run` method on the `client` object to execute the question-answer generation operation on the data shown above.

In [8]:
outputs = client.run(input_data)

  0%|          | 0/1 [00:00<?, ?it/s]/opt/conda/envs/uniflow/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
100%|██████████| 1/1 [00:27<00:00, 27.30s/it]


### Process the output

Let's take a look of the generated output, which is already a list of JSON

In [9]:
print("Sample output:")

pprint(outputs[0]['output'][0]['response'][0])

Sample output:
('instruction: Generate one question and its corresponding answer based on '
 'context. Following the format and structure of the examples below to include '
 'the same context, question, and answer in the response.\n'
 'context: In 1948, Claude E. Shannon published A Mathematical Theory of\n'
 'Communication (Shannon, 1948) establishing the theory of\n'
 'information. In his article, Shannon introduced the concept of\n'
 'information entropy for the first time. We will begin our journey here.\n'
 'question: Who published A Mathematical Theory of Communication in 1948?\n'
 'answer: Claude E. Shannon.\n'
 'current context: We believe our success depends upon our capabilities in '
 'areas such as design, research and development, production and marketing and '
 'is supported and protected by our intellectual property rights, such as '
 'trademarks, utility and design patents, copyrights, and trade secrets, among '
 'others. We have followed a policy of applying for and reg

In [10]:
import re

def extract_current_context(text):
    """Extracts the text after 'current context' and adds '**' to 'current context'."""
    match = re.search(r'(current context:.+)', text, re.DOTALL)
    if match:
        extracted_text = match.group(1)
        extracted_text = extracted_text.replace("current context", "**current context**")
        return extracted_text
    else:
        return "No match found."

for response in outputs[0]['output'][0]['response']:
    processed_response = extract_current_context(response)
    display(Markdown("---"))
    display(Markdown(processed_response))

---

**current context**: We believe our success depends upon our capabilities in areas such as design, research and development, production and marketing and is supported and protected by our intellectual property rights, such as trademarks, utility and design patents, copyrights, and trade secrets, among others. We have followed a policy of applying for and registering intellectual property rights in the United States and select foreign countries on trademarks, inventions, innovations and designs that we deem valuable. W e also continue to vigorously protect our intellectual property, including trademarks, patents and trade secrets against third-party infringement and misappropriation.

**Question:** What does current text suggest about protection IP Rights related with company's achievements

 **Answer**: The provided texts suggests  that Company has implemented policies designed specifically aimed at protecting their Intellectual Property(IP )Rights which includes trademark,utility & Design Patents,,copyrights&trade Secrets.The organization actively applies foe registration Of these right US And selected Foreign Countries while ensuring vigorous enforcement Against Infringement or Mis appropriation By Third Parties.. This strategy safeguards Their Achievements From Unauthorized Use Or Exploitation,. Therefore

---

**current context**: Snoopy can be selfish, gluttonous, and lazy at times, and occasionally mocks his owner, Charlie Brown. 
But on the whole, he shows great love, care, and loyalty for his owner (even though he cannot even remember 
his name and always refers to him as "the round-headed kid"). In the 1990s comic strips, he is obsessed 
with cookies, particularly the chocolate-chip variety. This, and other instances in which he indulges in large 
chocolate-based meals and snacks, indicate that chocolate is not poisonous to Snoopy, the way it is for real dogs.

**Question:** What does a passage describe about snoop'S relationship with food specifically chocolates

 **Answer**: The text describes Snoop’ s obsession over Chocolate Cookies  and indicates this treat isn t harmful like actual dog poison

---

**current context**: Since the 2000s and especially the late 2010s, Intel has faced increasing competition from AMD, resulting in a 
significant decline of its dominance and market share in the PC market. Nevertheless, with a 68.4% market share as of 
2023, Intel still leads the x86 market by a wide margin.

**Question:** What company is leading  the X-series processor marketplace currently despite facing increased competitive pressure since around year end two thousand or later specifically during Late twenty eleven'S era?

 **Answer**: Despite fierce rivalry against Advanced Micro Devices Inc.(AMD), intel maintains leadership position within this industry sector holding approximately sixty eight point four percent(or roughly seventy percentage )market coverage worldwide according data available through recent reports. Therefore,Intel remains dominant force even amidst growing challenges presented via competitors such As AmD

## End of the notebook

Check more Uniflow use cases in the [example folder](https://github.com/CambioML/uniflow/tree/main/example/model#examples)!

<a href="https://www.cambioml.com/" title="Title">
    <img src="../image/cambioml_logo_large.png" style="height: 100px; display: block; margin-left: auto; margin-right: auto;"/>
</a>